In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        display('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor() as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchall()[0]
                display(f"PostgreSQL database version: {db_version}")
                display(f"PostgreSQL connection status: {conn.info.status}")
                display(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        display(error)
        conn = None

In [3]:
import psycopg2

In [4]:
conn=init_connection(dsn)

'Connecting to the PostgreSQL database...'

"PostgreSQL database version: ('PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)"

'PostgreSQL connection status: 0'

"You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'."

In [5]:
from typing import List
def check_if_exist( context: List,conn):
        check_exist_SQL = """
                            SELECT
                                EXISTS (
                                    SELECT
                                        %s
                                    FROM
                                        public.dataset
                                    WHERE
                                        name = %s);
                        """
        exist_status = db_fetchone(check_exist_SQL, context,conn)[0]
        return exist_status

In [6]:
context=['name','asdfads']

In [5]:
def db_fetchone(sql_message, conn,vars: List=None):
    with conn:
        with conn.cursor() as cur:
            try:
                if vars:
                    cur.execute(sql_message, vars)
                else:
                    cur.execute(sql_message)
                conn.commit()
                return_one = cur.fetchone()  # return tuple
                return return_one
            except psycopg2.Error as e:
                display(e)

In [8]:
db_fetchone('select * from public.project',conn)

In [10]:
db_fetchone("select exists(select name from public.project where name = 'hello')",conn)

(False,)

In [11]:
context

['name', 'asdfads']

In [13]:
db_fetchone("select exists(select %s from public.project where name = %s)",conn,context)

(False,)

In [6]:
def db_fetchall(sql_message, conn,vars: List=None):
    with conn:
        with conn.cursor() as cur:
            try:
                if vars:
                    cur.execute(sql_message, vars)
                else:
                    cur.execute(sql_message)
                conn.commit()
                return_all = cur.fetchall()  # return tuple
                return return_all
            except psycopg2.Error as e:
                display(e)

In [21]:
query_dataset_SQL="""SELECT
    id,
    name,
    dataset_size,
    updated_at
FROM
    public.dataset;"""

dataset_list=db_fetchall(query_dataset_SQL,conn)
display(dataset_list)

[(1,
  'My First Dataset',
  None,
  datetime.datetime(2021, 7, 20, 11, 56, 43, 781535, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))),
 (2,
  'My Second Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 35, 18, 643911, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))),
 (3,
  'My Third Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 38, 58, 453171, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))]

In [125]:
updated_at=[]
dataset_name=[]
for dataset in dataset_list:
    without_timezone=dataset[3].strftime('%Y-%m-%d %H:%M:%S')
    updated_at.append(without_timezone)
    dataset_name.append(dataset[1])

display(dataset_name,updated_at)
    

['My First Dataset', 'My Second Dataset', 'My Third Dataset']

['2021-07-20 11:56:43', '2021-07-20 14:35:18', '2021-07-20 14:38:58']

In [23]:
from datetime import datetime

In [118]:
str(updated_at[0])

'2021-07-20 11:56:43.781535+08:00'

In [83]:
updated_at[0].tzinfo

psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)

In [119]:
updated_at[0].strftime("%d-%m-%Y %H:%M:%S")

'20-07-2021 11:56:43'

In [120]:
(updated_at[0].strftime('%Y-%m-%d %H:%M:%S'))

'2021-07-20 11:56:43'

In [8]:
import pandas as pd

In [130]:
df = pd.DataFrame( {'Dataset Name':dataset_name ,
    'Date':updated_at})
df

,Dataset Name,Date
0,My First Dataset,2021-07-20 11:56:43
1,My Second Dataset,2021-07-20 14:35:18
2,My Third Dataset,2021-07-20 14:38:58


In [ ]:
df['Date']=df['Date'].dt.tz_localize(None)


In [132]:

df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d %H:%M:%S')
df

,Dataset Name,Date
0,My First Dataset,2021-07-20 11:56:43
1,My Second Dataset,2021-07-20 14:35:18
2,My Third Dataset,2021-07-20 14:38:58


In [133]:
df.sort_values(by=['Date'],inplace=True,ascending=False,ignore_index=True)
df

,Dataset Name,Date
0,My Third Dataset,2021-07-20 14:38:58
1,My Second Dataset,2021-07-20 14:35:18
2,My First Dataset,2021-07-20 11:56:43


In [103]:
from copy import copy,deepcopy
query_dataset_SQL="""SELECT
    id,
    name,
    dataset_size,
    updated_at
FROM
    public.dataset;"""

dataset_list=db_fetchall(query_dataset_SQL,conn)
display(dataset_list)

# updated_at=[]
# dataset_name=[]
# for dataset in dataset_list:
#     without_timezone=dataset[3].strftime('%Y-%m-%d %H:%M:%S')
#     updated_at.append(without_timezone)
#     dataset_name.append(dataset[1])
datasets=list(dataset_list)
dataset_list=deepcopy(datasets)
datasets=[]
for dataset in dataset_list:
    dataset=list(dataset)

    # if dataset[2]:
        # dataset[2]=int(dataset[2])
    dataset[3]=dataset[3].strftime('%Y-%m-%d %H:%M:%S')
    datasets.append(dataset)
    
datasets
    





[(2,
  'My Second Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 35, 18, 643911, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))),
 (3,
  'My Third Dataset',
  4,
  datetime.datetime(2021, 7, 20, 14, 38, 58, 453171, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))]

[[2, 'My Second Dataset', 4, '2021-07-20 14:35:18'],
 [3, 'My Third Dataset', 4, '2021-07-20 14:38:58']]

In [117]:
df = pd.DataFrame( datasets,columns=['Dataset ID','Name','Dataset Size','Date/Time'])
# df=df.set_index('ID')
# df['Dataset Size']=df['Dataset Size'].astype('int32')
df.index.name=('No.')
df.dtypes
df.info()



df['Date/Time']=pd.to_datetime(df['Date/Time'],format='%Y-%m-%d %H:%M:%S')
df2=df.sort_values(by=['Date/Time'],ascending=False,ignore_index=True)
df2.index.name=('No.')
dfStyler = df2.style.set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
dfStyler





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Dataset ID    2 non-null      int64 
 1   Name          2 non-null      object
 2   Dataset Size  2 non-null      int64 
 3   Date/Time     2 non-null      object
dtypes: int64(2), object(2)
memory usage: 192.0+ bytes


,Dataset ID,Name,Dataset Size,Date/Time
No.,,,,
0,3,My Third Dataset,4,2021-07-20 14:38:58
1,2,My Second Dataset,4,2021-07-20 14:35:18


In [69]:
df2 = pd.DataFrame({'month': [1, 4, 7, 10],
                   'year': [2012, 2014, 2013, 2014]})

In [ ]:
df2=df2.set_index('year')


In [72]:
display(df2)

,month
year,
2012,1
2014,4
2013,7
2014,10


In [122]:
from queue import Queue
q=Queue()
for i in range(30):
    q.put(i)

print(q)
q.get_nowait()

0

In [129]:
db_fetchall("select name from public.dataset;",conn)

[('My Second Dataset',), ('My Third Dataset',)]

In [130]:
dataset_name=[]
for dataset in datasets:
    dataset_name.append(dataset[1])

dataset_name

['My Second Dataset', 'My Third Dataset']

In [149]:
from enum import Enum, IntEnum
class AnnotationType(IntEnum):
    Image_Classification = 1
    BBox = 2
    Polygons = 3
    Masks = 4
    def __str__(self):
        return self.name

    @classmethod
    def from_string(cls, s):
        try:
            return AnnotationType[s]
        except KeyError:
            raise ValueError()

In [150]:
AnnotationType.from_string('BBox')

<AnnotationType.BBox: 2>

# XML DOM 

In [2]:
import xml.etree.ElementTree as ET


In [8]:
data="""
<View>
<Header value="Select label and start to click on image"/>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
  </View>
</View>
"""

In [14]:
myroot=ET.fromstring(data)
print(myroot)
print(myroot.tag) #get root tag -> View tag
print(myroot.attrib) #check if there are attributes -> Dictionary

<Element 'View' at 0x7f12b9db7540>
View
{}


In [27]:
print(myroot[0].tag)#child tags
print(myroot[0].attrib)

Header
{'value': 'Select label and start to click on image'}


```
<View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
  </View>
  ```

In [29]:
print(myroot[1].tag)#child tags
print(myroot[1].attrib)

View
{'style': 'display:flex;align-items:start;gap:8px;flex-direction:column-reverse'}


In [34]:
#inside Outer View tag
for x in myroot[1]:
     print(x.tag, x.attrib)

Image {'name': 'img', 'value': '$image', 'zoom': 'true', 'zoomControl': 'true', 'rotateControl': 'false', 'grid': 'true', 'brightnessControl': 'true', 'contrastControl': 'true'}
View {}


In [38]:
# Inner View tag
for x in myroot[1][1]:
     print(x.tag, x.attrib)

Filter {'toName': 'tag', 'minlength': '0', 'name': 'filter'}
RectangleLabels {'name': 'tag', 'toName': 'img', 'showInline': 'true', 'fillOpacity': '0.5', 'strokeWidth': '5'}


In [41]:
# RectangleLabels tag
for x in myroot[1][1][1]:
     display(x.tag, x.attrib)

'Label'

{'value': 'Hello', 'background': 'blue'}

'Label'

{'value': 'World', 'background': 'pink'}

In [46]:
for x in myroot.findall('RectangleLabels'):
    Label=x.find('Label').text
    display(Label)

In [47]:
[elem.tag for elem in myroot.iter()]

['View',
 'Header',
 'View',
 'Image',
 'View',
 'Filter',
 'RectangleLabels',
 'Label',
 'Label']

In [66]:
def display_xml(xml_root):
    new_xml=ET.tostring(xml_root,encoding='unicode')
    print(new_xml)
    # return new_xml

In [72]:
def display_xml_utf_8(xml_root):
    new_xml=ET.tostring(xml_root,encoding='utf8').decode('utf-8')
    print(new_xml)
    # return new_xml

In [73]:
display_xml_utf_8(myroot)

<?xml version='1.0' encoding='utf8'?>
<View>
<Header value="Select label and start to click on image" />
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true" />
    <View>
      <Filter toName="tag" minlength="0" name="filter" />
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue" />
        <Label value="World" background="pink" />
      </RectangleLabels>
    </View>
  </View>
</View>


In [151]:
for label in myroot.iter('RectangleLabels'):
    print(label.tag,label.attrib)
    print(label)

RectangleLabels {'name': 'tag', 'toName': 'img', 'showInline': 'true', 'fillOpacity': '0.5', 'strokeWidth': '5'}
<Element 'RectangleLabels' at 0x7f12b95a3d10>


In [186]:
label_list=[]
for label in myroot.iter('Label'):
    print(label.tag,label.attrib)
    print(label)
    if label.attrib['value']=='aruco':
        label_list.append(label)
    else:pass
label_list[0].attrib

Label {'value': 'aruco', 'background': 'blue'}
<Element 'Label' at 0x7f12b95a3c20>
Label {'value': 'World', 'background': 'pink'}
<Element 'Label' at 0x7f12b95a3f40>


{'value': 'aruco', 'background': 'blue'}

In [101]:
for label in myroot.findall("./View/Image[@name='img']..."):#find attribute 
    print(label.attrib)

{'style': 'display:flex;align-items:start;gap:8px;flex-direction:column-reverse'}


In [168]:
for label in myroot[1][1][1].findall("./Label"):#find attribute 
    print(label.tag,label.attrib)
    

Label {'value': 'Hello', 'background': 'blue'}
Label {'value': 'World', 'background': 'pink'}


In [160]:
display(myroot[1][1][1].attrib)
display(myroot[1][1][1].get('name'))

{'name': 'tag',
 'toName': 'img',
 'showInline': 'true',
 'fillOpacity': '0.5',
 'strokeWidth': '5'}

'tag'

In [177]:
from copy import deepcopy
label=myroot[1][1][1].find("./Label")
print(label) #returns Element object
old_value=label.attrib['value']
print(old_value)
#change value of 'value'
label.attrib['value']='aruco'
label.attrib['value']
label.attrib

<Element 'Label' at 0x7f12b95a3c20>
aruco


{'value': 'aruco', 'background': 'blue'}

### After Modification

In [178]:
display_xml(myroot)

<View>
<Header value="Select label and start to click on image" />
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true" />
    <View>
      <Filter toName="tag" minlength="0" name="filter" />
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="aruco" background="blue" />
        <Label value="World" background="pink" />
      </RectangleLabels>
    </View>
  </View>
</View>


In [180]:
label_new=myroot[1][1][1].find("./Label[@value='aruco']")
print(label.attrib)

{'value': 'aruco', 'background': 'blue'}


## MINIDOM

In [352]:
from xml.dom import minidom
data="""
<View>
<Header value="Select label and start to click on image"/>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
  </View>
</View>
"""

In [340]:
doc=minidom.parseString(data)
doc

In [348]:
doc.getElementsByTagName('Label')[0].attributes.items()

[('value', 'Hello'), ('background', 'blue')]

In [359]:
doc.childNodes

[<DOM Element: View at 0x7f12b8cf9c10>]

## Create Node

In [374]:
def create_element(doc,parent,tagname,attr,value):
    nodeList = doc.getElementsByTagName(parent)[0]
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    new_label=doc.createElement(tagname)
    new_label.setAttribute(attr,value)
    
    nodeList.appendChild(new_label)

In [384]:
create_element(doc,'RectangleLabels','Label','value','Hello')

In [381]:
rectangle=doc.getElementsByTagName('RectangleLabels')[0]
print(rectangle)
new_label=doc.createElement('Label')
new_label.setAttribute("value","aruco")
new_label.setAttribute('background','red')
rectangle.appendChild(new_label)

<DOM Element: RectangleLabels at 0x7f12b891f940>


<DOM Element: Label at 0x7f12b891f700>

In [385]:
print(doc.toxml(encoding='utf8').decode('utf-8'))

<?xml version="1.0" encoding="utf8"?><View>
<Header value="Select label and start to click on image"/>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="aruco" background="blue"><Label value="Hello"/></Label>
        <Label value="World" background="pink"/>
      <Label value="aruco" background="red"/></RectangleLabels>
    </View>
  </View>
</View>


## Remove Nodes

In [367]:
def removeNodes(doc,tagname,attr,value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            parent = node.parentNode
            parent.removeChild(node)

## Edit Nodes

In [370]:
def set_attribute(doc,tagname,attr,value,new_value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            node.setAttribute(attr,new_value)

In [371]:
set_attribute(doc,'Label','value','Hello','aruco')

In [379]:
removeNodes(doc,'Label','value','Hello')

### Tutorial

In [190]:
tutorial_data='''<?xml version='1.0' encoding='utf8'?>
<collection>
    <genre category="Action">
        <decade years="1980s">
            <movie favorite="True" title="Indiana Jones: The raiders of the lost Ark">
                <format multiple="No">DVD</format>
                <year>1981</year>
                <rating>PG</rating>
                <description>
                'Archaeologist and adventurer Indiana Jones 
                is hired by the U.S. government to find the Ark of the 
                Covenant before the Nazis.'
                </description>
            </movie>
               <movie favorite="True" title="THE KARATE KID">
               <format multiple="Yes">DVD,Online</format>
               <year>1984</year>
               <rating>PG</rating>
               <description>None provided.</description>
            </movie>
            <movie favorite="False" title="Back 2 the Future">
               <format multiple="False">Blu-ray</format>
               <year>1985</year>
               <rating>PG</rating>
               <description>Marty McFly</description>
            </movie>
        </decade>
        <decade years="1990s">
            <movie favorite="False" title="X-Men">
               <format multiple="Yes">dvd, digital</format>
               <year>2000</year>
               <rating>PG-13</rating>
               <description>Two mutants come to a private academy for their kind whose resident superhero team must 
               oppose a terrorist organization with similar powers.</description>
            </movie>
            <movie favorite="True" title="Batman Returns">
               <format multiple="No">VHS</format>
               <year>1992</year>
               <rating>PG13</rating>
               <description>NA.</description>
            </movie>
               <movie favorite="False" title="Reservoir Dogs">
               <format multiple="No">Online</format>
               <year>1992</year>
               <rating>R</rating>
               <description>WhAtEvER I Want!!!?!</description>
            </movie>
        </decade>    
    </genre>

    <genre category="Thriller">
        <decade years="1970s">
            <movie favorite="False" title="ALIEN">
                <format multiple="Yes">DVD</format>
                <year>1979</year>
                <rating>R</rating>
                <description>"""""""""</description>
            </movie>
        </decade>
        <decade years="1980s">
            <movie favorite="True" title="Ferris Bueller's Day Off">
                <format multiple="No">DVD</format>
                <year>1986</year>
                <rating>PG13</rating>
                <description>Funny movie about a funny guy</description>
            </movie>
            <movie favorite="FALSE" title="American Psycho">
                <format multiple="No">blue-ray</format>
                <year>2000</year>
                <rating>Unrated</rating>
                <description>psychopathic Bateman</description>
            </movie>
        </decade>
    </genre>

    <genre category="Comedy">
        <decade years="1960s">
            <movie favorite="False" title="Batman: The Movie">
                <format multiple="Yes">DVD,VHS</format>
                <year>1966</year>
                <rating>PG</rating>
                <description>What a joke!</description>
            </movie>
        </decade>
        <decade years="2010s">
            <movie favorite="True" title="Easy A">
                <format multiple="No">DVD</format>
                <year>2010</year>
                <rating>PG--13</rating>
                <description>Emma Stone = Hester Prynne</description>
            </movie>
            <movie favorite="True" title="Dinner for SCHMUCKS">
                <format multiple="Yes">DVD,digital,Netflix</format>
                <year>2011</year>
                <rating>Unrated</rating>
                <description>Tim (Rudd) is a rising executive
                 who “succeeds” in finding the perfect guest, 
                 IRS employee Barry (Carell), for his boss’ monthly event, 
                 a so-called “dinner for idiots,” which offers certain 
                 advantages to the exec who shows up with the biggest buffoon.
                 </description>
            </movie>
        </decade>
        <decade years="1980s">
            <movie favorite="False" title="Ghostbusters">
                <format multiple="No">Online,VHS</format>
                <year>1984</year>
                <rating>PG</rating>
                <description>Who ya gonna call?</description>
            </movie>
        </decade>
        <decade years="1990s">
            <movie favorite="True" title="Robin Hood: Prince of Thieves">
                <format multiple="No">Blu_Ray</format>
                <year>1991</year>
                <rating>Unknown</rating>
                <description>Robin Hood slaying</description>
            </movie>
        </decade>
    </genre>
</collection>'''

In [191]:
tut_root=ET.fromstring(tutorial_data)

In [ ]:
list(tut_root.iter())

In [264]:
title=tut_root.find("./genre/decade/movie[@title='Back to the Future']")
list(title.iter())

[<Element 'movie' at 0x7f12b8db22c0>,
 <Element 'format' at 0x7f12b927f4f0>,
 <Element 'year' at 0x7f12b927f0e0>,
 <Element 'rating' at 0x7f12b9270c20>,
 <Element 'description' at 0x7f12b9ce4d10>]

In [199]:
wrong_title=tut_root.find("./genre/decade/movie[@title='Back 2 the Future']")
wrong_title.attrib['title']="Back to the Future"

In [261]:
for movie in tut_root.findall("./genre/decade/movie"):
    print(movie.attrib)

{'favorite': 'True', 'title': 'Indiana Jones: The raiders of the lost Ark'}
{'favorite': 'True', 'title': 'THE KARATE KID'}
{'favorite': 'False', 'title': 'Back to the Future'}
{'favorite': 'True', 'title': 'Batman Returns'}
{'favorite': 'False', 'title': 'Reservoir Dogs'}
{'favorite': 'False', 'title': 'X-Men'}
{'favorite': 'False', 'title': 'X-Men'}
{'favorite': 'False', 'title': 'ALIEN'}
{'favorite': 'True', 'title': "Ferris Bueller's Day Off"}
{'favorite': 'FALSE', 'title': 'American Psycho'}
{'favorite': 'False', 'title': 'Batman: The Movie'}
{'favorite': 'True', 'title': 'Easy A'}
{'favorite': 'True', 'title': 'Dinner for SCHMUCKS'}
{'favorite': 'False', 'title': 'Ghostbusters'}
{'favorite': 'True', 'title': 'Robin Hood: Prince of Thieves'}


In [ ]:
[elem.tag for elem in tut_root.iter()]

In [212]:
action=tut_root.find("./genre[@category='Action']")
print(action.attrib)
new_dec=ET.SubElement(action,'decade')
new_dec.attrib['years']='2000s'

{'category': 'Action'}


In [217]:
display_xml_utf_8(tut_root)

<?xml version='1.0' encoding='utf8'?>
<collection>
    <genre category="Action">
        <decade years="1980s">
            <movie favorite="True" title="Indiana Jones: The raiders of the lost Ark">
                <format multiple="No">DVD</format>
                <year>1981</year>
                <rating>PG</rating>
                <description>
                'Archaeologist and adventurer Indiana Jones 
                is hired by the U.S. government to find the Ark of the 
                Covenant before the Nazis.'
                </description>
            </movie>
               <movie favorite="True" title="THE KARATE KID">
               <format multiple="Yes">DVD,Online</format>
               <year>1984</year>
               <rating>PG</rating>
               <description>None provided.</description>
            </movie>
            <movie favorite="False" title="Back to the Future">
               <format multiple="False">Blu-ray</format>
               <year>1985</year>
 

In [215]:
xmen=tut_root.find("./genre/decade/movie[@title='X-Men']")
dec2000s=tut_root.find("./genre[@category='Action']/decade[@years='2000s']")
dec2000s.append(xmen)
dec1990s=tut_root.find("./genre[@category='Action']/decade[@years='1990s']")
dec1990s.remove(xmen)

In [227]:
third_data='''<?xml version="1.0" encoding="UTF-8"?>
<metadata>
<food>
    <item name="breakfast">Idly</item>
    <price>$2.5</price>
    <description>
   Two idly's with chutney
   </description>
    <calories>553</calories>
</food>
</metadata>
'''
third_root = ET.fromstring(third_data)

In [228]:
[elem.tag for elem in third_root.iter()]

['metadata', 'food', 'item', 'price', 'description', 'calories']

In [233]:
for item in third_root.iter('description'):
    print(item,item.text)

<Element 'description' at 0x7f12b8e6b720> 
   Two idly's with chutney
   


In [235]:
new_desc=str(item.text)+'will be served'
item.text=str(new_desc)
item.set('updated','yes')

In [237]:
display_xml_utf_8(third_root)

<?xml version='1.0' encoding='utf8'?>
<metadata>
<food>
    <item name="breakfast">Idly</item>
    <price>$2.5</price>
    <description updated="yes">
   Two idly's with chutney
   will be served</description>
    <calories>553</calories>
</food>
</metadata>


In [241]:
third_root[0] #food tag
ET.SubElement(third_root[0],'speciality') # create new tag
for x in third_root.iter('speciality'):
    new_desc='South Indian Special'
    x.text=str(new_desc)
display_xml(third_root)

<metadata>
<food>
    <item name="breakfast">Idly</item>
    <price>$2.5</price>
    <description updated="yes">
   Two idly's with chutney
   will be served</description>
    <calories>553</calories>
<speciality>South Indian Special</speciality></food>
</metadata>


In [254]:
# remove attribute
third_root[0][0].attrib.pop('name',None)

'breakfast'

In [255]:
#remove element
third_root[0].remove(third_root[0][0])
display_xml(third_root)

<metadata>
<food>
    <price>$2.5</price>
    <description updated="yes">
   Two idly's with chutney
   will be served</description>
    <calories>553</calories>
<speciality>South Indian Special</speciality></food>
</metadata>


In [258]:
for i in third_root[0].findall('calories'):
    print(i.text)

553


# xml.dom

In [266]:
from xml.dom import minidom

In [273]:
p3 = minidom.parseString(third_data)
p3

In [277]:
tagname= p3.getElementsByTagName('item')
print(tagname)
print(tagname[0].attributes['name'].value)

[<DOM Element: item at 0x7f12b88ae940>]
breakfast


In [279]:
print(tagname[0].firstChild.data)

Idly


In [296]:
p3.documentElement.tagName

'metadata'

In [289]:
data2="""<?xml version="1.0" ?>
<Floor floorNumber="1" floorRefId="723413320329068590" isOutdoor="false" name="DevNetZone" objectVersion="19">
    <Dimension height="16.5" length="81.9" offsetX="0.0" offsetY="0.0" unit="FEET" width="307.0"/>
    <Image imageName="domain_0_1421088463647.png"/>
    <GPSMarker name="GPS_Marker_17">
        <GeoCoordinate latitude="36.125859" longitude="-97.066969" unit="DEGREES"/>
        <MapCoordinate unit="FEET" x="0.6" y="0.6"/>
    </GPSMarker>
    <GPSMarker name="GPS_Marker_18">
        <GeoCoordinate latitude="36.125859" longitude="-97.06595" unit="DEGREES"/>
        <MapCoordinate unit="FEET" x="299.77" y="0.6"/>
    </GPSMarker>
    <GPSMarker name="GPS_Marker_19">
        <GeoCoordinate latitude="36.125641" longitude="-97.066969" unit="DEGREES"/>
        <MapCoordinate unit="FEET" x="0.6" y="80.09"/>
    </GPSMarker>
    <AccessPoint apMode="LOCAL" ethMacAddress="00:2b:01:00:04:f0" ipAddress="10.10.20.243" name="T1-3" numOfSlots="2" radioMacAddress="00:2b:01:00:04:00">
        <MapCoordinate unit="FEET" x="155.28" y="57.57"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="0" antennaPattern="Internal-1140-2.4GHz" band="IEEE_802_11_B" channelAssignment="1" channelNumber="1" slotNumber="0" txPowerLevel="1"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="11" antennaPattern="Internal-1140-5.0GHz" band="IEEE_802_11_A" channelAssignment="1" channelNumber="64" slotNumber="1" txPowerLevel="5"/>
    </AccessPoint>
    <AccessPoint apMode="LOCAL" ethMacAddress="00:2b:01:00:05:f0" ipAddress="10.10.20.244" name="T1-4" numOfSlots="2" radioMacAddress="00:2b:01:00:05:00">
        <MapCoordinate unit="FEET" x="213.6" y="12.6"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="0" antennaPattern="Internal-1140-2.4GHz" band="IEEE_802_11_B" channelAssignment="1" channelNumber="1" slotNumber="0" txPowerLevel="1"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="11" antennaPattern="Internal-1140-5.0GHz" band="IEEE_802_11_A" channelAssignment="1" channelNumber="64" slotNumber="1" txPowerLevel="5"/>
    </AccessPoint>
    <AccessPoint apMode="LOCAL" ethMacAddress="00:2b:01:00:06:f0" ipAddress="10.10.20.245" name="T1-5" numOfSlots="2" radioMacAddress="00:2b:01:00:06:00">
        <MapCoordinate unit="FEET" x="253.7" y="58.48"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="0" antennaPattern="Internal-1140-2.4GHz" band="IEEE_802_11_B" channelAssignment="1" channelNumber="1" slotNumber="0" txPowerLevel="1"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="11" antennaPattern="Internal-1140-5.0GHz" band="IEEE_802_11_A" channelAssignment="1" channelNumber="64" slotNumber="1" txPowerLevel="5"/>
    </AccessPoint>
    <AccessPoint apMode="LOCAL" ethMacAddress="00:2b:01:00:03:f0" ipAddress="10.10.20.242" name="T1-2" numOfSlots="2" radioMacAddress="00:2b:01:00:03:00">
        <MapCoordinate unit="FEET" x="98.1" y="11.7"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="0" antennaPattern="Internal-1140-2.4GHz" band="IEEE_802_11_B" channelAssignment="1" channelNumber="1" slotNumber="0" txPowerLevel="1"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="11" antennaPattern="Internal-1140-5.0GHz" band="IEEE_802_11_A" channelAssignment="1" channelNumber="64" slotNumber="1" txPowerLevel="5"/>
    </AccessPoint>
    <AccessPoint apMode="LOCAL" ethMacAddress="00:2b:01:00:02:f0" ipAddress="10.10.20.241" name="T1-1" numOfSlots="2" radioMacAddress="00:2b:01:00:02:00">
        <MapCoordinate unit="FEET" x="43.9" y="57.88"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="0" antennaPattern="Internal-1140-2.4GHz" band="IEEE_802_11_B" channelAssignment="1" channelNumber="1" slotNumber="0" txPowerLevel="1"/>
        <ApInterface antennaAngle="1.57" antennaElevAngle="0.0" antennaGain="11" antennaPattern="Internal-1140-5.0GHz" band="IEEE_802_11_A" channelAssignment="1" channelNumber="64" slotNumber="1" txPowerLevel="5"/>
    </AccessPoint>
    <LocationFilterRegion regionType="OUTSIDE">
        <MapCoordinate unit="FEET" x="0.0" y="0.0"/>
        <MapCoordinate unit="FEET" x="307.0" y="0.0"/>
        <MapCoordinate unit="FEET" x="307.0" y="81.9"/>
        <MapCoordinate unit="FEET" x="0.0" y="81.9"/>
    </LocationFilterRegion>
</Floor>"""

In [297]:
data_root=minidom.parseString(data2)
data_root.documentElement.tagName

'Floor'

In [294]:
data_root.getElementsByTagName('Floor')

[<DOM Element: Floor at 0x7f12b88aeb80>]

```
<AccessPoint apMode="LOCAL" ethMacAddress="00:2b:01:00:03:f0" ipAddress="10.10.20.242" name="T1-2" numOfSlots="2" radioMacAddress="00:2b:01:00:03:00"></AccessPoint>
```

In [309]:
access_points=data_root.getElementsByTagName('AccessPoint')
access_points


[<DOM Element: AccessPoint at 0x7f12b88f5160>,
 <DOM Element: AccessPoint at 0x7f12b88f5820>,
 <DOM Element: AccessPoint at 0x7f12b88f5a60>,
 <DOM Element: AccessPoint at 0x7f12b88f5ca0>,
 <DOM Element: AccessPoint at 0x7f12b88f5ee0>]

In [316]:
for access_point in access_points:
    ap_name=access_point.getAttribute('name')
    ap_mac=access_point.getAttribute('ethMacAddress')
    ap_ip=access_point.getAttribute('ipAddress')
    print(access_point.tagName + ': ' + ap_name + '\t mac: ' + ap_mac + '\t ip: '+ ap_ip)

AccessPoint: T1-3	 mac: 00:2b:01:00:04:f0	 ip: 10.10.20.243
AccessPoint: T1-4	 mac: 00:2b:01:00:05:f0	 ip: 10.10.20.244
AccessPoint: T1-5	 mac: 00:2b:01:00:06:f0	 ip: 10.10.20.245
AccessPoint: T1-2	 mac: 00:2b:01:00:03:f0	 ip: 10.10.20.242
AccessPoint: T1-1	 mac: 00:2b:01:00:02:f0	 ip: 10.10.20.241


In [318]:
data_root.getElementsByTagName('MapCoordinate')[0].attributes.items()

[('unit', 'FEET'), ('x', '0.6'), ('y', '0.6')]